# Gesture Recognition

Dependencies  
`pip install tensorflow`  
`pip install tensorflow-object-detection-api`  
`pip install opencv-python`

In [1]:
import os
import time
import numpy as np
import cv2

## Setting up the paths

In [2]:
WORKPLACE_PATH = 'Tensorflow/workplace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKPLACE_PATH + '/annotations'
IMAGE_PATH = WORKPLACE_PATH + '/images'
DATA_PATH = IMAGE_PATH + '/allimages'
MODEL_PATH = WORKPLACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKPLACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

## Making Dataset

numbers: both hands, front + back = 40 * 5 = 200  
left indicate: both hands = 40  
right indicate: both hands = 40  
namaste = 40  
adaab = 40  
salute = 40  
eating = 40  
**TOTAL = 440 * 2 = 880**

In [3]:
labels = ['one', 'two', 'three', 'four', 'five', 'leftIndicate',
          'rightIndicate', 'namaste', 'adaab', 'salute', 'eating']
number_images = 40

In [ ]:
for label in labels:
    print(f'Collecting images for {label}')
    path = os.path.join(DATA_PATH, label)
    capture = cv2.VideoCapture(0)
    time.sleep(5)

    i = 0
    while True:
        ret, frame = capture.read()
        cv2.imshow('video', frame)

        name = f'k-{label}-frame' + str(i) + '.jpg'
        
        if i % 7 == 0:
            cv2.imwrite(os.path.join(path, name), frame)

        i += 1  # incrementing count
        
        if i == 7*number_images:
            break

        # video will run infinitely till 'q' is pressed on keyboard
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # release the capture and closing the video window after end
    capture.release()
    cv2.destroyAllWindows()

## Generate Labels

In [8]:
labels = [
    {'name':'one', 'id':1},
    {'name':'two', 'id':2},
    {'name':'three', 'id':3},
    {'name':'four', 'id':4},
    {'name':'five', 'id':5},
    {'name':'leftIndicate', 'id':6},
    {'name':'rightIndicate', 'id':7},
    {'name':'namaste', 'id':8},
    {'name':'adaab', 'id':9},
    {'name':'salute', 'id':10},
    {'name':'eating', 'id':11},
]

In [9]:
# with open(os.path.join(ANNOTATION_PATH, 'label_map.pbtxt'), 'w') as f:
#     for label in labels:
#         f.write('item { \n')
#         f.write('\tname:\'{}\'\n'.format(label['name']))
#         f.write('\tid:{}\n'.format(label['id']))
#         f.write('}\n')
# print('Generated label map')

Generated label map


## Generate TF records

In [ ]:
if os.name == 'posix':
    !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
    !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}
else:
    !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
    !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

## Download TF models

In [ ]:
# !cd Tensorflow && git clone https://github.com/tensorflow/models

In [ ]:
# !cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## Setting up pipeline config

In [ ]:
# !mkdir {MODEL_PATH + CUSTOM_MODEL_NAME}
# !cp {PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

**model.ssd.num_classes** = 11

**train_config.batch_size** = 4

**train_config.fine_tune_checkpoint** = 'Tensorflow/workplace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

**train_config.fine_tune_checkpoint_type** = "detection"

**train_input_reader.label_map_path** = ANNOTATION_PATH + '/label_map.pbtxt'

**train_input_reader.tf_record_input_reader.input_path[:]** = [ANNOTATION_PATH + '/train.record']

**eval_input_reader[0].label_map_path** = ANNOTATION_PATH + '/label_map.pbtxt'

**eval_input_reader[0].tf_record_input_reader.input_path[:]** = [ANNOTATION_PATH + '/test.record']

## Training the model

In [ ]:
num = 150000

In [ ]:
print("--- for posix ---")
print(f"""python3 {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps={num}""")

print("\n--- else ---")
print(f"""python {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps={num}""")